# This notebook goes through the procedure of MR and behavioral data processing.
## First - import packages and load data

In [33]:
#!pip install numpy pandas matplotlib seaborn sklearn
from pathlib import Path
import numpy as np
import pandas as pd

data_directory = Path('../data')
mr_data = pd.read_pickle(data_directory / 'mr_data.pickle')
ims_data = pd.read_pickle(data_directory / 'ims_data.pickle')

In [34]:
for i,row in mr_data.iterrows():
    closest_ims_row = abs(row["session"] - ims_data["datetime"]).idxmin()
    mr_data.loc[i, "ims_degree"] = ims_data.loc[closest_ims_row, "Degrees"]

In [37]:

from sklearn import preprocessing
columns_to_scale = ["ims_degree","age","height","weight","hour"]
for col in columns_to_scale:
    mr_data[f"{col}_scaled"] = preprocessing.scale(mr_data[col].values)
    mr_data[f"{col}_MC"] = mr_data[col] - mr_data[col].mean()

     |████████████████████████████████| 30.8 MB 13.2 MB/s eta 0:00:01


In [236]:
import os
CMD_TEMPLATE = "rsync --exclude '*IRT*' -azp --progress zvi@132.66.42.126:/media/hdd/media/MRI/rawdata/{subject} /media/groot/Minerva/Covid19/MRI/rawdata/"
base_dir = Path("/media/groot/Minerva/Covid19/MRI/rawdata")
for subj_dir in base_dir.glob("sub-*"):
    subject = subj_dir.name
    cmd = CMD_TEMPLATE.format(subject=subject)
    # print(cmd)
    os.system(cmd)




receiving incremental file list
sub-258/ses-201908141739/anat/
receiving incremental file list
sub-211/
sub-211/ses-201912150926/fmap/sub-211_ses-201912150926_acq-dwi_dir-REV_epi.json
          4,108 100%    3.92MB/s    0:00:00 (xfr#1, to-chk=75/98)
sub-211/ses-201912150926/func/sub-211_ses-201912150926_task-RfmriRestApSbref_dir-FWD_sbref.json
          3,614 100%    3.45MB/s    0:00:00 (xfr#2, to-chk=73/98)
sub-211/ses-201912150926/func/sub-211_ses-201912150926_task-RfmriRestApSbref_dir-FWD_sbref.nii.gz
      1,305,024 100%  207.43MB/s    0:00:00 (xfr#3, to-chk=72/98)
sub-211/ses-201912150926/func/sub-211_ses-201912150926_task-RfmriRestAp_dir-FWD_bold.json
          3,625 100%  590.01kB/s    0:00:00 (xfr#4, to-chk=71/98)
sub-211/ses-201912150926/func/sub-211_ses-201912150926_task-RfmriRestAp_dir-FWD_bold.nii.gz
    893,002,614 100%  475.51MB/s    0:00:01 (xfr#5, to-chk=70/98)
sub-211/ses-202005131711/dwi/
sub-211/ses-202005131711/dwi/sub-211_ses-202005131711_dir-FWD_run-1_dwi.bval
   

In [228]:
group = "climbing"
condition = "learner"
cohort = "old"
mask = mr_data.T.any()
if group:
    mask = (mask) & (mr_data["group"] == group)
if condition:
    mask = (mask) & (mr_data["condition"] == condition)
if cohort:
    mask = (mask) & (mr_data["cohort"] == cohort)
subset = mr_data[mask]
valid_participants = subset.groupby("participant").count()["session"] > 1
subset = subset.set_index("participant")[valid_participants].reset_index()
md = MixedLM(subset["58"],subset[["timedelta"]],subset["participant"],exog_re=subset[["age","weight","height","hour","ims_degree"]]).fit(method="lbfgs")
md.summary()

/tmp/ipykernel_321246/604299824.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset = subset.set_index("participant")[valid_participants].reset_index()
/home/groot/Projects/PhD/papers/real-life-plasticity/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/groot/Projects/PhD/papers/real-life-plasticity/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/groot/Projects/PhD/papers/real-life-plasticity/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2705: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))


<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:               MixedLM   Dependent Variable:   58      
No. Observations:    31        Method:               REML    
No. Groups:          14        Scale:                0.0000  
Min. group size:     2         Log-Likelihood:       226.1623
Max. group size:     3         Converged:            Yes     
Mean group size:     2.2                                     
-------------------------------------------------------------
                        Coef.  Std.Err. z P>|z| [0.025 0.975]
-------------------------------------------------------------
timedelta                0.000                               
age Var                  0.000    0.006                      
age x weight Cov        -0.000                               
weight Var               0.000    0.002                      
age x height Cov        -0.000    0.001                      
weight x height Cov      0.000    0.001                      
height Var               0.000    0.001                      
age x hour Cov           0.000    0.000                      
weight x hour Cov        0.000    0.000                      
height x hour Cov        0.000    0.000                      
hour Var                 0.000    0.000                      
age x ims_degree Cov    -0.000    0.000                      
weight x ims_degree Cov -0.000    0.000                      
height x ims_degree Cov -0.000                               
hour x ims_degree Cov    0.000    0.000                      
ims_degree Var           0.000    0.000                      
=============================================================

"""

In [229]:

subset

,participant,session,1,2,3,4,5,6,7,8,...,ims_degree_scaled,ims_degree_MC,age_scaled,age_MC,height_scaled,height_MC,weight_scaled,weight_MC,hour_scaled,hour_MC
0,119,2020-11-26 14:03:00,0.001082,0.000873,0.000742,0.000787,0.000819,0.000761,0.000715,0.000943,...,-1.005654,-4.736441,1.088653,5.643478,-0.818320,-7.730435,-0.816059,-8.452174,-0.055274,-0.177966
1,119,2021-05-05 14:45:00,0.001059,0.000880,0.000820,0.000862,0.000957,0.000861,0.000738,0.000965,...,0.820322,3.863559,1.088653,5.643478,-0.818320,-7.730435,-0.816059,-8.452174,-0.055274,-0.177966
2,40,2020-12-29 08:43:00,0.001019,0.000803,0.000687,0.000815,0.000833,0.000851,0.000794,0.000928,...,-0.496080,-2.336441,0.124130,0.643478,0.875391,8.269565,0.535644,5.547826,-1.918787,-6.177966
3,40,2021-03-17 13:14:00,0.001040,0.000829,0.000700,0.000823,0.000846,0.000835,0.000810,0.000929,...,-0.432383,-2.036441,0.124130,0.643478,0.875391,8.269565,0.535644,5.547826,-0.365859,-1.177966
4,40,2021-07-06 10:48:00,0.001022,0.000837,0.000705,0.000828,0.000915,0.000892,0.000832,0.000911,...,1.457290,6.863559,0.124130,0.643478,0.875391,8.269565,0.535644,5.547826,-1.297616,-4.177966
5,43,2021-04-25 12:37:00,0.001146,0.000964,0.000781,0.000854,0.000963,0.000943,0.000880,0.000915,...,-0.156363,-0.736441,-0.840393,-4.356522,0.875391,8.269565,1.114945,11.547826,-0.676445,-2.177966
6,43,2021-07-08 13:15:00,0.001084,0.000968,0.000777,0.000842,0.000949,0.000905,0.000871,0.000902,...,1.839471,8.663559,-0.840393,-4.356522,0.875391,8.269565,1.114945,11.547826,-0.365859,-1.177966
7,46,2021-03-30 10:11:00,0.000863,0.000924,0.000747,0.000727,0.000690,0.000760,0.000851,0.000904,...,-0.835796,-3.936441,0.895749,4.643478,-1.241748,-11.730435,-0.912609,-9.452174,-1.297616,-4.177966
8,46,2021-07-05 18:37:00,0.000847,0.000973,0.000765,0.000733,0.000761,0.000862,0.000861,0.000902,...,1.053877,4.963559,0.895749,4.643478,-1.241748,-11.730435,-0.912609,-9.452174,1.187068,3.822034
9,7,2020-11-29 17:38:00,0.001063,0.000913,0.000737,0.000880,0.000925,0.000841,0.000838,0.000919,...,-0.857028,-4.036441,0.509939,2.643478,0.557820,5.269565,-0.140208,-1.452174,0.876483,2.822034
